# 04. CODE REPRESENTATION

1. [x] введение
2. [x] CodeBERT
5. [x] упражнение
6. [x] ссылки

# 1. Введение

Гипотеза (например, [Hindle et al - On the naturalness of software 2016](https://dl.acm.org/doi/abs/10.1145/2902362)):
>
> *Язык программирования (PL) похож на естественный язык (NL).*

Если так, то можем использовать эффективные подходы из NLP, учитывая специфику PL.

Encoder-only:
- CuBERT (Kanade et al, 2019)
- CodeBERT (Feng et al, 2020)
- GraphCodeBERT (Guo et al, 2020)
- SynCoBERT (Wang et al, 2022)

Decoder-only:
- GPT-C (Svyatkovskiy et al, 2020)
- CodeGPT (Lu et al, 2021)

Encoder-decoder:
- PLBART (Ahmad et al, 2021)
- CodeT5 (Wang et al, 2021)
- TreeBERT (Jiang et al, 2021)
- UniXcoder (Guo et al, 2022)

# 2. CodeBERT

[CodeBERT](https://arxiv.org/abs/2002.08155v4) --- бимодальная предварительно обученная модель для языков программирования и естественного языка. CodeBERT на выходе выдаёт эмбеддинги общего назначения. Эмбеддинги могут использоваться в таких задач, как поиск по коду, суммаризация кода и т. д. Модель разработана в Microsoft в 2020 году.

![](./res/04_codebert_paper.png)

## Архитектура

В основе модели CodeBERT лежит [BERT](https://arxiv.org/abs/1810.04805)(Google, 2018). Точнее, [RoBERTa](https://arxiv.org/abs/1907.11692) (Meta, 2019, отличие в процедуре формирования масок).

![](https://miro.medium.com/max/720/1*p4LFBwyHtCw_Qq9paDampA.png)

In [ ]:
import pprint
import torch
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pp = pprint.PrettyPrinter(indent=2)

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")

In [ ]:
nl_tokens=tokenizer.tokenize("return maximum value")
print(nl_tokens)

In [ ]:
code_tokens=tokenizer.tokenize("def max(a,b): if a>b: return a else return b")
print(code_tokens)

In [ ]:
tokens=[tokenizer.cls_token]+nl_tokens+[tokenizer.sep_token]+code_tokens+[tokenizer.sep_token]
print(tokens)

In [ ]:
tokens_ids=tokenizer.convert_tokens_to_ids(tokens)
print(tokens_ids)

In [ ]:
len(tokens_ids)

In [ ]:
context_embeddings=model(torch.tensor(tokens_ids)[None,:])[0]

In [ ]:
context_embeddings.shape

In [ ]:
print(context_embeddings)

## Pre-training


CodeBERT --- это бимодальная модель. Это значит, что на вход могут подаваться пары вида (NL, PL).
При этом пара преобразуется в последовательность токенов.
Для этого, каждый элемент пары: NL- и PL-часть, с помощью токенизатора ([Byte Pair Encoding](https://en.wikipedia.org/wiki/Byte_pair_encoding), BPE) превращается в последовательность токенов (сегмент).

На этапе предобучения модели на вход подаётся конкатенация двух сегментов со специальным токеном-разделителем:

$$[\texttt{CLS}], w_1, w_2, \ldots, w_n, [\texttt{SEP}], c_1, c_2, \ldots, c_m, [\texttt{EOS}].$$

- первый сегмент: текст естественного языка
- второй сегмент: исходный код

$[\texttt{CLS}]$ --- специальный токен для получение агрегированного эмбеддинга (может быть использован, например, при классификации)

На выходе имеем:
- контекстное векторное представление каждого токена (для естественного языка и кода);
- представление для токена $[\texttt{CLS}]$.


### Данные

Обучение модели CodeBERT происходит как на *бимодальных* данных (параллельные пары текст и кода), так и на *унимодальных* данных.

- из репозиториев Github
- бимодальная пара --- это отдельная функция и соответствующая документация (примерно, 2 млн)
- унимодальные данные --- это функция без парной документации (примерно, 6 млн)
- шесть языков программирования: Python, Java, JavaScript, PHP, Ruby и Go.

### Обучение

Предобучение происходит на двух задачах:
- MLM (masked language modeling), BERT, Devlin et al. (2018): восстановление токенов, бимодальные данные
- RTD (replaced token detection), ELECTRA, Clark et al. (2020): обнаружение замены токенов, унимодальные данные

**Задача MLM**

![](https://raw.githubusercontent.com/UKPLab/sentence-transformers/master/docs/img/MLM.png)

Пусть $x = \{w, c\}$ --- входная NL-PL пара, где $w$ --- последовательность NL-токенов, а $c$ --- последовательность PL-токенов.

Сначала выбираем случайный набор позиций для NL и PL, которые необходимо замаскировать --- $m_w$ и $m_c$ соответственно. Затем заменяем выбранные позиции специальным токеном $[\texttt{MASK}]$. Как и в Devlin et al. (2018), маскируется 15% токенов из $x$ (если выбран токен, то с вероятностью 0.8 он заменяется на $[\texttt{MASK}]$, с вероятностью 0.1 заменяется на случайный токен, с вероятностью 0.1 остаётся прежним).

- для $i \in \{1, \ldots, |w|\}$ имеем $m^w_i ∼ \texttt{unif}\{1, |w|\}$ --- какие NL-токены хотим маскировать
- для $i \in \{1, \ldots, |c|\}$ имеем $w^c_m ∼ \texttt{unif}\{1, |c|\}$ --- какие PL-токены хотим маскировать
- $w^{masked} = \texttt{REPLACE}(w, m^w, [\texttt{MASK}])$
- $c^{masked} = \texttt{REPLACE}(c, m^c, [\texttt{MASK}])$
- $x = [w;c]$ --- конкатенируем

В MLM задача состоит в том, чтобы предсказать токены, которые были маскированы. Для этого используется следующая функция потерь:

$$L_{MLM}(\theta) = \sum_{i \in m^w \cup m^c} -\log p (x_i~|~w^{masked}, c^{masked}).$$

In [ ]:
from transformers import RobertaConfig, RobertaTokenizer, RobertaForMaskedLM, pipeline

model = RobertaForMaskedLM.from_pretrained("microsoft/codebert-base-mlm")
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base-mlm")

In [ ]:
CODE = """
if x is not <mask>:
    x += 1"
"""
fill_mask = pipeline('fill-mask', model=model, tokenizer=tokenizer)

outputs = fill_mask(CODE)
pp.pprint(outputs)

**Задача RTD**

Изначально, задача RTD (replaced token detection) была использована для предобучния моделей естественного языка [ELECTRA](https://arxiv.org/abs/2003.10555). Для модели CodeBERT эта задача была адаптирована для кода.
Используются как *бимодальные*, так и *унимодальные* данные. В частности, здесь есть два генератора данных:
- NL-генератор $p^{G_w}$ и
- PL-генератор $p^{G_c}$.

Оба генератора нужны для создания правдоподобных альтернатив выбранных токенов.

![](./res/04_rtd_electra.png)

Генератор --- произвольая языковая модель, генерирующая распределение на токенах. Обычно используют небольшую  модель, обучающуюся совместно с генератором.

Более формально:
- для $i \in m^w$ имеем $\hat{w}_i ∼ p^{G_w} (w_i~|~w_{masked})$ --- какие токены и на какие хотим заменить
- для $i \in m^c$ имеем $\hat{c}_i ∼ p^{G_c} (c_i~|~c_{masked})$ --- аналогично для кода
- $w^{corrupt} = \texttt{REPLACE}(w, m^w , \hat{w})$
- $c^{corrupt} = \texttt{REPLACE}(c, m^c , \hat{c})$
- $x^{corrupt} = [w^{corrupt};c^{corrupt}]$ --- конкатенируем 

Задача дискриминатора определить, является ли токен оригинальным или нет, т.е. бинарная классификация.
![](./res/04_rtd_codebert.png)

Заметим, что генератор применяется к каждой позиции во входных данных и отличается от GAN тем,
 что, если генератор выдает правильный токен, то метка этого токена "original", а не "replaced".


Функция потерь для задачи RTD:

$$L_{RTD}(\theta) = \sum_{i=1}^{|w|+|c|}(\delta(i) \log p^{D} (x^{corrupt}, i) + (1-\delta(i))(1-\log p^{D}(x^{corrupt}, i))),$$

где
- $\theta$ --- параметры,
- $p^{D}$ --- дискриминатор, предсказывающий вероятность того, что $i$-й токен является оригинальным,
- $\delta(i)$ --- индикаторная функция: $\delta(i) = 1$, если $x^{corrupt}_i = x_i$; иначе, $\delta(i) = 0.$

Существует множество различных способов реализации генераторов. Для CodeBERT используются две модели языка (NL, PL) на основе  $n$-грамм (Jurafsky, 2000) с двунаправленными контекстами. Модели были обучены на соответствующих унимодальных данных.

**Задача MLM+RTD**

Итоговая задача выглядит так:
$$ \min_{\theta}( L_{MLM}(\theta) + L_{RTD}(\theta) )$$

##  Реализация

- https://github.com/microsoft/CodeBERT
- https://huggingface.co/microsoft/graphcodebert-base

# 5. Упражнение

Исследовать возможность использовать CodeBERT для задачи clone detection (поиск клонов).
Например, можно использовать датасет [BigCloneBench](https://github.com/clonebench/BigCloneBench)

# 6 Полезные ссылки

- [Kanade et al - CuBERT: Learning and Evaluating Contextual Embedding of Source Code](https://arxiv.org/abs/2001.00059)
- [Feng et al - CodeBERT: A Pre-trained Model for Programming and Natural Languages 2020](https://arxiv.org/abs/2002.08155)
- [Karampatsis Sutton - SCELMo: Source Code Embeddings from Language Models 2020](https://arxiv.org/abs/2004.13214)
- [Jain et al - Contrastive Code Representation Learning 2020](https://arxiv.org/abs/2007.04973)
- [Guo et al - GraphCodeBERT: Pre-training Code Representations with Data Flow 2020](https://arxiv.org/abs/2009.08366)
- [Chirkova Torshin - An Empirical Study of Transformers for Source Code 2020](https://arxiv.org/abs/2010.07987)
- [Gu et al - Multimodal representation for neural code search 2022](https://arxiv.org/abs/2107.00992)
- [Guo et al - UniXcoder: Unified Cross-Modal Pre-training for Code Representation](https://arxiv.org/abs/2203.03850)